ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [1]:

import json

#GETTING MY ID:
id = open('../id_CONFIDENTIAL.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]
#my_id

In [1]:
!python3 -m pip install pip==24.0



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\avril\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install azureml-sdk

  Using cached azureml_sdk-1.59.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached azureml_core-1.59.0.post2-py3-none-any.whl.metadata (3.4 kB)
  Using cached azureml_dataset_runtime-1.59.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached azureml_train_core-1.59.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached azureml_train_automl_client-1.59.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached azureml_pipeline-1.59.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached msal-1.32.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached knack-0.12.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
  U


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
## STEP 1: Set up the workspace.


from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = my_id,
                      resource_group = "resource_group2",
                      location = "centralindia")



Error: Azure ML was unable to load cloud metadata from the url specified by https://management.azure.com/metadata/endpoints?api-version=2019-05-01. HTTPSConnectionPool(host='management.azure.com', port=443): Max retries exceeded with url: /metadata/endpoints?api-version=2019-05-01 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E881495E10>: Failed to resolve 'management.azure.com' ([Errno 11001] getaddrinfo failed)")). This may be due to a misconfiguration of networking controls. Azure Machine Learning Python SDK requires outbound access to Azure Resource Manager. Please contact your networking team to configure outbound access to Azure Resource Manager on both Network Security Group and Firewall. For more details on required configurations, see https://docs.microsoft.com/azure/machine-learning/how-to-access-azureml-behind-firewall.


KeyboardInterrupt: 

In [2]:
from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="../best_model.pkl",
                                  model_name=mname,
                                  workspace=ws)

NameError: name 'ws' is not defined

In [ ]:
from azureml.core.model import Model

mname_scaler = "model_scaler"
registered_scaler = Model.register(model_path="../scaler.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [15]:
################ score.py ###################

scorepy = f"""
import json
import joblib
import pandas as pd
from azureml.core.model import Model
from zorrouno import processor


def init():
    global model
    global scaler
    # Obtener la ruta de los modelos
    model_path = Model.get_model_path('{mname}')
    model = joblib.load(model_path)
    
    scaler_path = Model.get_model_path('{mname_scaler}')
    scaler = joblib.load(scaler_path)


def run(raw_data):
    try:
        # Cargar los datos de entrada
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)
        
        # Procesar los datos (embeddings o transformaciones previas)
        embedded_data = processor.embbed(data)
        
        # Escalar los datos utilizando el scaler
        scaled_data = scaler.transform(embedded_data)
        
        # Realizar la predicción con el modelo
        result_finals = model.predict(scaled_data)
        
        # Devolver el resultado en formato JSON
        return json.dumps(result_finals.tolist())  # Convertir a lista para serialización
    except Exception as e:
        return json.dumps(str(e))

"""

# Guardar el script en un archivo
file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()


In [ ]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-bank")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn', 'joblib']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model, registered_scaler],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

/tmp/ipykernel_54428/944137596.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [11]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
